In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [ ]:
df_patients_22 = pd.read_csv("2021VAERSData/2021VAERSDATA.csv", index_col='VAERS_ID', encoding='latin1', low_memory=False)

In [ ]:
df_patients_22.index
df_patients_22

In [ ]:
df_vax_22 = pd.read_csv("2021VAERSData/2021VAERSVAX.csv", index_col='VAERS_ID', encoding='latin1')

In [ ]:
df_vax_22

In [ ]:
df_COVID19_22 = df_vax_22[df_vax_22["VAX_TYPE"] == "COVID19"]

In [ ]:
df_COVID19_22

In [ ]:
df_patients_COVID19 = pd.merge(df_patients_22, df_COVID19_22, left_index=True, right_index=True)

In [ ]:
df_patients_22.shape, df_COVID19_22.shape, df_patients_COVID19.shape

In [ ]:
# pd.set_option('display.max_rows', 100)
df_patients_COVID19

In [ ]:
df_PFIZER_BIONTECH = df_patients_COVID19[df_patients_COVID19["VAX_MANU"] == "MODERNA"]

In [ ]:
df_PFIZER_BIONTECH.info()

In [ ]:
# table = pd.pivot_table(df_PFIZER_BIONTECH, values='DIED', columns=['DIED'], aggfunc=np.sum)

In [ ]:
df = df_PFIZER_BIONTECH[df_PFIZER_BIONTECH['DIED']=='Y'][['VAX_LOT']]
df

In [ ]:
# df = df.groupby(['VAX_LOT']).count()
#df2 = df.sort_values(by=['DIED'], ascending=False)
#pd.set_option('display.max_rows', None)
#print(df2)
pd.set_option('display.max_rows', None)
df.value_counts()